In [1]:
%load_ext jupyter_black

# nvidia cuda


``` bash
nvidia-smi --query-gpu=timestamp,name,pci.bus_id,driver_version,pstate,pcie.link.gen.max,pcie.link.gen.current,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv
nvidia-smi --query-gpu=name,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv -l 5

nvidia-smi -q -g 0 -d UTILIZATION -l
==============NVSMI LOG==============

Timestamp                                 : Sat Aug 13 18:07:02 2022
Driver Version                            : 516.59
CUDA Version                              : 11.7

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Utilization
        Gpu                               : 1 %
        Memory                            : 1 %
        Encoder                           : 0 %
        Decoder                           : 0 %
    GPU Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 6 %
        Min                               : 0 %
        Avg                               : 0 %
    Memory Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 1 %
        Min                               : 0 %
        Avg                               : 0 %
    ENC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
    DEC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
```

In [1]:
import distributed

from dask_cuda import LocalCUDACluster



def close():
    """
    for convenience if a cluster/client is already running
    this should close them to prevent starting multiple clusters
    on the same port.
    """
    try:
        client.close(), cluster.close()
    except:
        ...


close()
cluster = LocalCUDACluster()
client = distributed.Client(cluster)

client

/opt/conda/envs/rapids/lib/python3.9/site-packages/dask_cuda/utils.py:244: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(
2022-08-14 16:29:24,037 - distributed.diskutils - INFO - Found stale lock file and directory '/workspaces/griblib/notebooks/probsevere/dask-worker-space/worker-b__xr6md', purging
2022-08-14 16:29:24,038 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 15.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34969,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 15.57 GiB
Comm: tcp://127.0.0.1:34439,Total threads: 1
Dashboard: http://127.0.0.1:40645/status,Memory: 15.57 GiB
Nanny: tcp://127.0.0.1:46311,


In [14]:
import os
from pathlib import Path
import pandas as pd
import dask.dataframe as dd
import cudf

# creating a Path object from the absolute path to our data folder
ps_data = Path(os.path.abspath("probsevere-bucket"))
all_files = tuple(ps_data.rglob("*.pq"))
ps_data

PosixPath('/workspaces/griblib/notebooks/probsevere/probsevere-bucket')

In [18]:
len(all_files)

2023

In [21]:
%%time
# reading with pure pandas and pyarrow
pd.concat((pd.read_parquet(file,engine="pyarrow") for file in all_files))

CPU times: user 13.3 s, sys: 2.93 s, total: 16.2 s
Wall time: 12.4 s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,CENTROID_X,CENTROID_Y
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
30,3004,2349,-3,35.099998,110,30.799999,0.00,0.43,1,0.05,...,-9.376000,3,77643,2022-04-07 17:20:38,-81.080002,28.459999,-80.839996,28.639999,-80.952003,28.546778
30,2070,1924,0,33.900002,177,41.500000,0.00,0.61,0,0.00,...,-8.539000,2,76622,2022-04-07 12:28:36,-81.970001,30.129999,-81.889999,30.209999,-81.924965,30.170076
30,47,9,-999,8.200000,137,43.200001,0.00,0.52,0,0.00,...,16.847000,0,79257,2022-04-07 23:22:37,-75.540001,42.599998,-75.250000,42.900002,-75.385941,42.722469
30,324,79,-29,38.200001,85,23.700001,0.06,1.01,2,0.06,...,18.365999,2,73398,2022-04-07 03:00:40,-81.470001,37.250000,-81.269997,37.480000,-81.361763,37.362385
30,2452,1983,-9,21.700001,61,30.799999,0.13,0.78,2,0.09,...,-8.779000,1,74932,2022-04-07 06:38:38,-75.709999,33.650002,-75.629997,33.720001,-75.668236,33.681961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1387,811,-27,57.599998,64,10.300000,0.00,0.55,0,0.00,...,11.488000,6,25158,2022-04-03 04:24:36,-82.250000,24.930000,-82.180000,25.000000,-82.221428,24.965477
17,2433,1577,-59,52.200001,26,20.100000,0.16,0.63,6,0.12,...,0.339000,18,24782,2022-04-03 00:34:39,-78.900002,27.320000,-78.790001,27.480000,-78.839378,27.389109
17,1623,853,-8,54.000000,35,7.500000,0.00,0.46,0,0.02,...,-0.803000,7,25211,2022-04-03 05:30:36,-81.820000,24.639999,-81.739998,24.700001,-81.781418,24.669413


In [23]:
%%time
# much faster to read into a dask dataframe
ddf: dd.DataFrame = dd.read_parquet(all_files)
ddf

CPU times: user 93.7 ms, sys: 7.46 ms, total: 101 ms
Wall time: 95.2 ms


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,MAXLLAZ,P98LLAZ,P98MLAZ,WETBULB_0C_HGT,PWAT,CAPE_M10M30,LJA,SIZE,MOTION_EAST,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,CENTROID_X,CENTROID_Y
npartitions=2023,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,uint32,int32,float32,uint32,float32,float32,float32,uint32,float32,float32,float32,float32,float32,float32,uint32,float32,uint32,float32,float32,uint8,uint32,datetime64[ns],float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [24]:
%%time
# again, but the compute time to convert it back to a pandas native object is more expensive
ddf: dd.DataFrame = dd.read_parquet(all_files)
ddf.compute()

CPU times: user 8.8 s, sys: 584 ms, total: 9.38 s
Wall time: 21.8 s


,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,MOTION_SOUTH,PS,ID,VALIDTIME,MINX,MINY,MAXX,MAXY,CENTROID_X,CENTROID_Y
0,1524,1270,0,51.400002,36,17.900000,0.92,2.08,36,1.25,...,6.203000,88,136524,2022-03-01 00:00:29,-79.209999,30.120001,-79.070000,30.280001,-79.137489,30.199476
0,61,74,0,11.100000,225,46.000000,0.00,0.45,0,0.00,...,2.775000,0,137200,2022-03-01 10:42:34,-122.099998,45.380001,-121.949997,45.509998,-122.020500,45.443287
0,74,76,0,12.200000,227,45.599998,0.00,0.45,0,0.00,...,-0.083000,0,137200,2022-03-01 10:40:35,-122.169998,45.330002,-121.949997,45.500000,-122.061707,45.428375
0,990,642,0,32.299999,27,13.300000,0.10,0.70,0,0.00,...,-0.477000,2,137196,2022-03-01 10:38:37,-78.800003,26.600000,-78.709999,26.709999,-78.757126,26.650797
0,990,642,0,32.299999,27,13.300000,0.11,0.72,0,0.00,...,38.986000,2,137196,2022-03-01 10:36:32,-78.800003,26.600000,-78.709999,26.709999,-78.757126,26.650797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,3153,2232,-6,51.299999,53,13.900000,0.37,0.73,5,0.17,...,-4.856000,27,73167,2022-04-07 01:42:40,-78.050003,28.299999,-77.970001,28.389999,-78.010391,28.343540
123,2170,1127,-75,22.299999,147,29.799999,0.00,0.54,0,0.00,...,-10.151000,1,73787,2022-04-07 03:14:39,-83.000000,32.070000,-82.940002,32.139999,-82.971207,32.101852
124,1901,1557,-39,36.299999,98,22.299999,0.00,0.41,0,0.00,...,20.238001,3,73778,2022-04-07 03:12:39,-87.750000,31.389999,-87.669998,31.480000,-87.715858,31.432455
124,2015,1783,-29,20.700001,157,34.299999,0.05,0.66,3,0.09,...,18.540001,1,73788,2022-04-07 03:14:39,-80.300003,31.959999,-80.209999,32.040001,-80.253624,31.995804


In [26]:
%%time
# the parquet files we're create using dask.dataframe
gdf: cudf.DataFrame = cudf.read_parquet(all_files)
# gdf[["ID","FLASH_RATE","PS"]] = gdf[["ID","FLASH_RATE","PS"]].astype(int)
# df = gdf.set_index(["VALIDTIME", "ID"]).sort_index().to_pandas()

del gdf  # delete the dataframe from gpu memory after it has been loaded to ram

CPU times: user 1.01 s, sys: 363 ms, total: 1.37 s
Wall time: 1.42 s


In [34]:
%%time
# the parquet files we're create using dask.dataframe
def read_probsevere(paths:tuple[Path,...])->cudf.DataFrame: 
    return cudf.read_parquet(paths).set_index(["VALIDTIME","ID"]).sort_index()
gdf = read_probsevere(all_files)
df = gdf.to_pandas()
del gdf
df

CPU times: user 1.09 s, sys: 557 ms, total: 1.65 s
Wall time: 1.74 s


MUCAPE  MLCAPE  MLCIN    EBSHEAR  SRH01KM  \
VALIDTIME           ID                                                  
2022-03-01 00:00:29 136524    1524    1270      0  51.400002       36   
                    136565     756     433      0  39.200001       82   
                    136592     384     112     -5  37.900002       80   
                    136648     551     422     -4  33.299999       24   
                    136649      10       0   -999   6.400000      170   
...                            ...     ...    ...        ...      ...   
2022-04-07 23:58:34 79444      121       8   -999  12.100000      328   
                    79445      200      10   -999  11.900000      422   
                    79446      505      10   -999  43.599998      100   
                    79447     1917    1603   -999  54.400002       86   
                    79448     2910    2078   -999  42.099998       45   

                            MEANWIND_1-3kmAGL  MESH  VIL_DENSITY  FLASH_RATE  \
VALIDTIME           ID                                                         
2022-03-01 00:00:29 136524          17.900000  0.92         2.08          36   
                    136565          16.200001  0.11         0.68           0   
                    136592           7.300000  0.00         0.78           0   
                    136648          18.000000  0.08         0.94           0   
                    136649          39.000000  0.00         0.27           0   
...                                       ...   ...          ...         ...   
2022-04-07 23:58:34 79444           55.299999  0.00         0.26           0   
                    79445           53.599998  0.00         0.47           0   
                    79446           29.200001  0.00         0.46           0   
                    79447           33.299999  0.00         0.50           0   
                    79448           19.200001  0.07         0.37           0   

                            FLASH_DENSITY  ...  SIZE  MOTION_EAST  \
VALIDTIME           ID                     ...                      
2022-03-01 00:00:29 136524           1.25  ...   201        6.493   
                    136565           0.00  ...    55        8.784   
                    136592           0.00  ...   271        5.375   
                    136648           0.02  ...    83        6.686   
                    136649           0.00  ...   119        5.171   
...                                   ...  ...   ...          ...   
2022-04-07 23:58:34 79444            0.00  ...    84        6.797   
                    79445            0.00  ...    63        6.843   
                    79446            0.00  ...    60        7.403   
                    79447            0.00  ...   123       15.852   
                    79448            0.03  ...   346        8.298   

                            MOTION_SOUTH  PS        MINX       MINY  \
VALIDTIME           ID                                                
2022-03-01 00:00:29 136524         6.203  88  -79.209999  30.120001   
                    136565         5.193   4  -80.320000  28.020000   
                    136592         5.236   2  -80.949997  29.990000   
                    136648         5.148   2  -81.129997  28.309999   
                    136649         4.775   0 -123.760002  47.470001   
...                                  ...  ..         ...        ...   
2022-04-07 23:58:34 79444        -10.562   0  -74.540001  42.590000   
                    79445        -11.285   0  -74.769997  42.070000   
                    79446        -16.528   3  -76.260002  38.779999   
                    79447        -13.444   7  -76.519997  36.110001   
                    79448         -3.296   8  -84.330002  24.809999   

                                  MAXX       MAXY  CENTROID_X  CENTROID_Y  
VALIDTIME           ID                                                     
2022-03-01 00:00:29 136524  -79.070000  30.280001  -79.137489  